In [48]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from PIL import Image, ImageFont, ImageDraw
import numpy as np
import string
import sys
import os
from skimage.io import imread
from sklearn.model_selection import ShuffleSplit
from TFANN import ANNC

import tensorflow as tf

In [156]:
driver = webdriver.Chrome('/home/lucas.cardoso/fastfinger-tensorflow/chromedriver_linux64/chromedriver')
driver.get('https://10fastfingers.com/login')

In [1]:
login = input("Email: ")
password = input ("Password: ")

KeyboardInterrupt: 

In [ ]:
driver.find_element('id', 'UserEmail').send_keys(login)
driver.find_element('id', 'UserPassword').send_keys(password)
driver.find_element('id', 'login-form-submit').click()

In [ ]:
while True:
    try:
        highlightWord = driver.find_elements_by_class_name("highlight")[0]   
    except:
        break
    driver.find_element('id','inputfield').send_keys(highlightWord.text + " ")
    time.sleep(0.07)

In [ ]:
driver.get("https://10fastfingers.com/anticheat/view/1/1")
driver.find_element('id', 'start-btn').click()

In [ ]:
imageUrl = driver.find_elements_by_tag_name('img')[0];
driver.save_screenshot("screenshot.png")

In [ ]:
from PIL import Image
imageObject  = Image.open("screenshot.png")
cropped = imageObject.crop((300,190,880,320))
cropped.save("screenshot.png")

In [46]:
def MakeImg(t, f, fn, s = (100, 100), o = (16, 8)):
    img = Image.new('RGB', s, "black")
    draw = ImageDraw.Draw(img)
    draw.text(OFS, t, (255, 255, 255), font = f)
    img.save(fn)
    
CS = list(string.ascii_letters) + list(string.digits)

RTS = list(np.random.randint(10, 64, size = 8192)) + [64]
S = [''.join(np.random.choice(CS, i)) for i in RTS]

font = ImageFont.truetype("LiberationMono-Regular.ttf", 16)

MS = max(font.getsize(Si) for Si in S)

OFS = ((640 - MS[0]) // 2, (32 - MS[1]) // 2)

MS = (640, 32)
Y = []
for i, Si in enumerate(S):
    MakeImg(Si, font, str(i) + '.png', MS, OFS)
    Y.append(str(i) + '.png,' + Si)

with open('Train.csv', 'w') as F:
    F.write('\n'.join(Y))
    
def LoadData(FP = '.'):
    TFP = os.path.join(FP, 'Train.csv')
    A, Y, T, FN = [], [], [], []
    with open(TFP) as F:
        for Li in F:
            FNi, Yi = Li.strip().split(',')                     
            T.append(Yi)
            A.append(imread(os.path.join(FP, 'Out', FNi)))
            Y.append(list(Yi) + [' '] * (MAX_CHAR - len(Yi)))   
            FN.append(FNi)
    return np.stack(A), np.stack(Y), np.stack(T), np.stack(FN)

    #Architecture of the neural network
    NC = len(string.ascii_letters + '')
    MAX_CHAR = 64
    IS = (14, 640, 3)       #Image size for CNN
    ws = [('C', [4, 4,  3, NC // 2], [1, 2, 2, 1]), ('AF', 'relu'), 
          ('C', [4, 4, NC // 2, NC], [1, 2, 1, 1]), ('AF', 'relu'), 
          ('C', [8, 5, NC, NC], [1, 8, 5, 1]), ('AF', 'relu'),
          ('R', [-1, 64, NC])]
    #Create the neural network in TensorFlow
    cnnc = ANNC(IS, ws, batchSize = 64, learnRate = 5e-5, maxIter = 32, reg = 1e-5, tol = 1e-2, verbose = True)
    if not cnnc.RestoreModel('TFModel/', 'ocrnet'): cnnc.fit(A, Y)

    YH = np.zeros((Y.shape[0], Y.shape[1]), dtype = np.int)
    for i in np.array_split(np.arange(A.shape[0]), 32): 
        YH[i] = np.argmax(cnnc.predict(A[i]), axis = 2)

    PS = [''.join(CS[j] for j in YHi) for YHi in YH]
    for PSi, Ti in zip(PS, T):
        print(Ti + '\t->\t' + PSi)


In [49]:
kernel = tf.Variable(tf.truncated_normal([k, k, chIn, chOut], stddev=0.1))
conv = tf.nn.conv2d(inputTensor, kernel, padding='SAME', strides=(1, 1, 1, 1))

NameError: name 'k' is not defined